In [90]:
from transformers import pipeline
import os
import pandas as pd
from datasets import Dataset
import evaluate

model_id = "facebook/bart-large-mnli"

path = r"D:\Grad School - VT\Coursework\8th Semester (F24, CPE)\Natural Language Processing\Project"

df = pd.read_parquet(os.path.join(path, "filtered.parquet"))
df = Dataset.from_pandas(df)
df

Dataset({
    features: ['CaseId', 'Summary', 'MaxVehicleSeverity', 'MaxOccupantInjurySeverity'],
    num_rows: 17584
})

In [91]:
classifier = pipeline("zero-shot-classification", model=model_id, device=0)

zeroshot_preds = classifier(df['Summary'], ['Minor', 'Moderate', 'Severe'])

zeroshot_preds[1]

{'sequence': 'V1 was traveling north in lane 1 of a 2 lane divided roadway. V1 departed the road to the left colliding with the guardrail. V1 then returned to the roadway and began rolling across the two lanes and departed the roadway again to the right side. V1 came to rest to the east of the roadway on all four wheels.',
 'labels': ['Severe', 'Moderate', 'Minor'],
 'scores': [0.4532017111778259, 0.35471948981285095, 0.19207872450351715]}

In [92]:
print(model_id, "model Zeroshot Metrics:")
# Convert predictions to a format suitable for evaluate
references = []
predicted_labels = []

label_mapping = {'Minor': 0, 'Moderate': 1, 'Severe': 2}

for prediction in zeroshot_preds:
    # Assuming the label with the highest score is the prediction
    predicted_label_index = prediction["scores"].index(max(prediction["scores"]))
    predicted_label = prediction["labels"][predicted_label_index]

    # Map the predicted label to its numerical equivalent
    predicted_labels.append(label_mapping[predicted_label])

accuracy_metric = evaluate.load("accuracy")
accuracy_results = accuracy_metric.compute(predictions=predicted_labels, references=df['MaxVehicleSeverity'])
print(accuracy_results)

precision_metric = evaluate.load("precision")
precision_results = precision_metric.compute(predictions=predicted_labels, references=df['MaxVehicleSeverity'], average='weighted', zero_division=0) 
print(precision_results)

recall_metric = evaluate.load("recall")
recall_results = recall_metric.compute(predictions=predicted_labels, references=df['MaxVehicleSeverity'], average='weighted') 
print(recall_results)

f1_metric = evaluate.load("f1")
f1_results = f1_metric.compute(predictions=predicted_labels, references=df['MaxVehicleSeverity'], average='weighted') 
print(f1_results)

facebook/bart-large-mnli model Zeroshot Metrics:
{'accuracy': 0.411794813466788}
{'precision': 0.33284989973614937}
{'recall': 0.411794813466788}
{'f1': 0.3652391536603057}


In [93]:
fuc = pd.read_json(os.path.join(path, "case_25540.json"))

fuc.columns

ValueError: All arrays must be of the same length